In [ ]:
#Library Import
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import time
import statistics
import math



In [220]:
#Execution Time
start_time = time.time()

#Read Dataset
df = pd.read_excel('49FC.xlsx')

#Drop unnecessary columns
df1 = df.drop(['SehirAdi','merchaltgrupkod','Buyergruptanim',
               'ModelKod','RenkKod','UrunAd','RenkTanim','beden','boy','Toplam veri: Brutkar'
              ,'Toplam veri: DevirHizi','Toplam veri: IndirimOran','Toplam veri: BrutkarDM3'
              ],axis = 1)

maxWeek = df1.merchyilhafta.max()
#Filtering Dataset for the last 3 weeks MA preperation
df1 = df1[df1['merchyilhafta'] <= maxWeek]
df1 = df1[df1['merchyilhafta'] >= (maxWeek - 2)]
df1['urunklasmanTanim'] = df1['urunklasmanTanim'].astype(str)
#df['Toplam veri: SatisAdet'] = df['Toplam veri: SatisAdet'].astype(str)

#To Prevent Sum Dates in Groupby, Type Changing on Years
df1['merchyilhafta'] = df['merchyilhafta'].astype(str)

#Groupby wrt 4 columns
initial = df1.groupby(['Magaza','SKU','merchyilhafta','urunklasmanTanim']).sum().reset_index()


#Rename complex column's name
initial = initial.rename(columns = {'Toplam veri: SatisAdet' : 'Sales'})
initial = initial.rename(columns = {'Magaza' : 'Stores'})
initial = initial.rename(columns = {'merchyilhafta' : 'Weeks'})
initial = initial.rename(columns = {'urunklasmanTanim' : 'Category'})
initial = initial.rename(columns = {'Gun Sonu Toplam Stok': 'Initial_Stock'})
initial = initial.rename(columns = {'Toplam veri: Fiyat': 'Price'})


In [221]:
#To Prevent Sum Dates in Groupby, Type Changing on Years
#Change Years to numerical value

initialv2 = initial
initialv2['Weeks'] = initialv2['Weeks'].astype(int)
initialv2['Sales'] = initialv2['Sales'].astype(int)
initialv2['Price'] = initialv2['Price'].astype(float)
initialv2['IW'] = initialv2['Initial_Stock'].astype(int)

In [222]:
lenData = len(initialv2) - 1
lastStore = initialv2['Stores'][lenData-1]
lastSKU = initialv2['SKU'][lenData-1]
lastWeek = initialv2['Weeks'][lenData-1]
lastCategory = initialv2['Category'][lenData-1]

In [223]:
#Function to fill empty Dates

for i in range(1, 6194):
    
    if((initialv2['Stores'][i-1] == initialv2['Stores'][i] == initialv2['Stores'][i+1]) & 
       (initialv2['SKU'][i-1] == initialv2['SKU'][i] == initialv2['SKU'][i+1]) & 
       (initialv2['Weeks'][i-1]+2 == initialv2['Weeks'][i]+1 == initialv2['Weeks'][i+1] == maxWeek)):
            i += 2
            continue
            
            
    
    elif((initialv2['Stores'][i-1] == initialv2['Stores'][i]) & 
       (initialv2['SKU'][i-1] == initialv2['SKU'][i])):
        
    #if 2 sales are exist and last week missing, fill the last week.
        if(((initialv2['Weeks'][i-1]) == (initialv2['Weeks'][i] - 1)) & 
           (initialv2['Weeks'][i] == maxWeek -1 )):
            
            
             new_row = {'Stores' : initialv2['Stores'][i-1], 'SKU' : initialv2['SKU'][i-1],
                      'Weeks' : maxWeek , 'Sales' : 0,'Category': initialv2['Category'][i-1],
                        'Initial_Stock': initialv2['Initial_Stock'][i],
                        'Price' : initialv2['Price'][i],'Wmin' : initialv2['Wmin'][i],
                       'Wmax' : initialv2['Wmax'][i],'IW' : initialv2['IW'][i]}
    
             initialv2 = initialv2.append(new_row, ignore_index = True)
             initialv2 = initialv2.groupby(['Stores','SKU','Weeks','Category']).sum().reset_index()
             i = i+1
    
        elif ((initialv2['Weeks'][i-1]) == (initialv2['Weeks'][i] - 1)):
            i = i+1
            continue
        elif ((initialv2['Weeks'][i-1]) != (initialv2['Weeks'][i] - 1)):
            if ((initialv2['Weeks'][i-1] > 201753) | 
                (initialv2['Weeks'][i-1] > 201853) | (initialv2['Weeks'][i-1] > 201953)):
                i = i+1
                continue
            else:    
                new_row = {'Stores' : initialv2['Stores'][i-1], 'SKU' : initialv2['SKU'][i-1],
                      'Weeks' : initialv2['Weeks'][i-1]+1, 'Sales' : 0,
                        'Category': initialv2['Category'][i-1],
                        'Initial_Stock': initialv2['Initial_Stock'][i-1],
                        'Price' : initialv2['Price'][i-1],'Wmin' : initialv2['Wmin'][i-1],
                       'Wmax' : initialv2['Wmax'][i-1],'IW' : initialv2['IW'][i-1]}
                initialv2 = initialv2.append(new_row, ignore_index = True)
                initialv2 = initialv2.groupby(['Stores','SKU','Weeks','Category']).sum().reset_index()
                i = i-1
         
    else:
        i = i+1
        continue
        

In [224]:
initialv2.tail()

,Stores,SKU,Weeks,Category,Initial_Stock,Sales,Price,Wmin,Wmax,IW
6190,UA20,7K9189Z8_A1J_L,201748,KEY TRIKO KAZAK,1,1,54.297856,2,6,1
6191,UA20,7K9189Z8_A1J_M,201747,KEY TRIKO KAZAK,2,2,54.297856,2,6,2
6192,UA20,7KI586Z8_HFH_L,201748,KEY TRIKO KAZAK,3,2,49.593361,2,6,3
6193,UA20,7KI586Z8_HFH_M,201748,KEY TRIKO KAZAK,3,2,49.593361,2,6,3
6194,UA20,7KI586Z8_HFH_S,201748,KEY TRIKO KAZAK,2,2,49.593361,2,6,2


In [225]:
#Creating Final Input
dataset = initialv2.groupby(['Stores','SKU','Weeks','Category']).sum().reset_index()
dummydata = dataset

#Adding Forecast Column to Dataset
dataset['Forecast'] = 0
dataset['Std'] = 0
dataset['Lower'] = 0
dataset['Upper'] = 0
#Moving Average Parameters for n = 2 and n = 3 ,
i=0

n = 3
t = 4.302653

nn = 2
tt = 12.70620


In [226]:
lenData2 = len(dataset) - 1
lastStore2 = dataset['Stores'][lenData2]
lastSKU2 = dataset['SKU'][lenData2]
lastWeek2 = dataset['Weeks'][lenData2]
lastCategory2 = dataset['Category'][lenData2]

In [227]:
#MA CALCULATION

while i < 10714: 
    #print(dataset[:(i+1)])
    #print(i)
    #if((dataset['Stores'][i+1] == lastStore2) & (dataset['Category'][i+1] == lastCategory2) & 
       #(dataset['Weeks'][i+1] == lastWeek2) & (dataset['SKU'][i] == lastSKU2)):
        #i = 60000
        #continue
    
    #Check if only 1 sale exists of the SKU i
    if((dataset['SKU'][i] != dataset['SKU'][i+1])):
        # 1 sku MA wrt n = 3
        if(dataset['Weeks'][i] == maxWeek-2):
            
            new_row = {'Stores' : dataset['Stores'][i], 'SKU' : dataset['SKU'][i],
                       'Weeks' : maxWeek + 1, 'Sales' : 0, 'Forecast' : 0,'Std' : 0, 'Lower' : 0,
                      'Upper' : 0, 'Category': dataset['Category'][i],
                        'Initial_Stock': dataset['Initial_Stock'][i],
                        'Price' : dataset['Price'][i],'Wmin' : dataset['Wmin'][i],
                       'Wmax' : dataset['Wmax'][i],'IW' : dataset['IW'][i]}
            
            dataset = dataset.append(new_row, ignore_index = True)
            dataset = dataset.groupby(['Stores','SKU','Weeks','Category']).sum().reset_index() 
            dataset['Forecast'][i+1] = round((dataset['Sales'][i] / 3))
            
            stdd = np.std(([dataset['Sales'][i], 0, 0]), ddof=1)
            dataset['Std'][i+1] = stdd
            dataset['Upper'][i+1] =  round(dataset['Forecast'][i+3] + 
                                           (t *(dataset['Std'][i+3] / math.sqrt(n))))
        
            dataset['Lower'][i+3] =  round(dataset['Forecast'][i+3] - 
                                           (t *(dataset['Std'][i+3] / math.sqrt(n))))
            
            if (dataset['Lower'][i+3] < 0):
                dataset['Lower'][i+3]  = 0             
            
            i = i+2
            
          # 1 sku MA wrt n = 2
        elif(dataset['Weeks'][i] == maxWeek-1):
            
            new_row = {'Stores' : dataset['Stores'][i], 'SKU' : dataset['SKU'][i],
                       'Weeks' : maxWeek + 1, 'Sales' : 0,'Forecast' : 0,'Std' : 0, 
                       'Lower' : 0, 'Upper' : 0, 'Category': dataset['Category'][i],
                        'Initial_Stock': dataset['Initial_Stock'][i],
                        'Price' : dataset['Price'][i],'Wmin' : dataset['Wmin'][i],
                       'Wmax' : dataset['Wmax'][i],'IW' : dataset['IW'][i]}
            
            dataset = dataset.append(new_row, ignore_index = True)
            dataset = dataset.groupby(['Stores','SKU','Weeks','Category']).sum().reset_index() 
            dataset['Forecast'][i+1] = round((dataset['Sales'][i] / 2))
            stdd = np.std(([dataset['Sales'][i], 0]), ddof=1)
            dataset['Std'][i+1] = stdd
            dataset['Upper'][i+1] =  round(dataset['Forecast'][i+1] + 
                                           (tt *(dataset['Std'][i+1] / math.sqrt(nn))))
        
            dataset['Lower'][i+1] =  round(dataset['Forecast'][i+1] - 
                                           (tt *(dataset['Std'][i+1] / math.sqrt(nn))))
            
            if (dataset['Lower'][i+1] < 0):
                dataset['Lower'][i+1]  = 0             
            
            i = i+2
           
            # 1 sku MA wrt n = 1 / same value
        elif(dataset['Weeks'][i] == maxWeek):
            
            new_row = {'Stores' : dataset['Stores'][i], 'SKU' : dataset['SKU'][i],
                       'Weeks' : maxWeek + 1, 'Sales' : 0, 'Forecast' : 0,'Std' : 0, 'Lower' : 0,
                      'Upper' : 0, 'Category': dataset['Category'][i],
                        'Initial_Stock': dataset['Initial_Stock'][i],
                        'Price' : dataset['Price'][i],'Wmin' : dataset['Wmin'][i],
                       'Wmax' : dataset['Wmax'][i],'IW' : dataset['IW'][i]}
            
            dataset = dataset.append(new_row, ignore_index = True)
            dataset = dataset.groupby(['Stores','SKU','Weeks','Category']).sum().reset_index() 
            dataset['Forecast'][i+1] = dataset['Sales'][i]
            stdd = 0
            dataset['Std'][i+1] = stdd
            dataset['Upper'][i+1] =  round(dataset['Forecast'][i+1] + 
                                           (tt *(dataset['Std'][i+1] / math.sqrt(nn))))
        
            dataset['Lower'][i+1] =  round(dataset['Forecast'][i+1] - 
                                           (tt *(dataset['Std'][i+1] / math.sqrt(nn))))
            
            if (dataset['Lower'][i+1] < 0):
                dataset['Lower'][i+1]  = 0             
            
            i = i+2
           
        else:
            i+1
                                              
        #MA n = 3
    elif((dataset['SKU'][i]==dataset['SKU'][i+1]==dataset['SKU'][i+2])&
       (dataset['Weeks'][i]+2==dataset['Weeks'][i+1]+1==dataset['Weeks'][i+2] == maxWeek)):
           
            new_row = {'Stores' : dataset['Stores'][i+2], 'SKU' : dataset['SKU'][i+2],
                       'Weeks' : dataset['Weeks'][i+2]+1, 'Sales' : 0, 'Forecast' : 0,'Std' : 0, 
                       'Lower' : 0, 'Upper' : 0, 'Category': dataset['Category'][i+2],
                        'Initial_Stock': dataset['Initial_Stock'][i+2],
                        'Price' : dataset['Price'][i+2],'Wmin' : dataset['Wmin'][i+2],
                       'Wmax' : dataset['Wmax'][i+2],'IW' : dataset['IW'][i+2]}
            
            dataset = dataset.append(new_row, ignore_index = True)
            dataset = dataset.groupby(['Stores','SKU','Weeks','Category']).sum().reset_index()      
            dataset['Forecast'][i+3] = round((dataset['Sales'][i] + 
                                              dataset['Sales'][i+1] + dataset['Sales'][i+2])/3)
            
            stdd = np.std(([dataset['Sales'][i], dataset['Sales'][i+1], 
                            dataset['Sales'][i+2]]), ddof=1)
            
            dataset['Std'][i+3] = stdd
            
            dataset['Upper'][i+3] =  round(dataset['Forecast'][i+3] + 
                                           (t *(dataset['Std'][i+3] / math.sqrt(n))))
            
            dataset['Lower'][i+3] =  round(dataset['Forecast'][i+3] - 
                                           (t *(dataset['Std'][i+3] / math.sqrt(n))))
            
            if (dataset['Lower'][i+3] < 0):
                dataset['Lower'][i+3]  = 0             
            
            i = i+4
         
        #MA n = 2
    elif((dataset['SKU'][i]==dataset['SKU'][i+1])&
         (dataset['Weeks'][i]+1==dataset['Weeks'][i+1] == maxWeek)):
        
            new_row = {'Stores' : dataset['Stores'][i+1], 'SKU' : dataset['SKU'][i+1],
                       'Weeks' : dataset['Weeks'][i+1]+1, 'Sales' : 0, 'Forecast' : 0,'Std' : 0, 'Lower' : 0,
                      'Upper' : 0, 'Category': dataset['Category'][i+1],
                        'Initial_Stock': dataset['Initial_Stock'][i+1],
                        'Price' : dataset['Price'][i+1],'Wmin' : dataset['Wmin'][i+1],
                       'Wmax' : dataset['Wmax'][i+1],'IW' : dataset['IW'][i+1]}
            
            dataset = dataset.append(new_row, ignore_index = True)
            
            dataset = dataset.groupby(['Stores','SKU','Weeks','Category']).sum().reset_index()
            dataset['Forecast'][i+2] = round((dataset['Sales'][i] + dataset['Sales'][i+1])/2)
            
            stdddd = np.std(([dataset['Sales'][i], dataset['Sales'][i+1], 
                              dataset['Sales'][i+2]]), ddof=1)
            
            dataset['Std'][i+2] = stdddd
            
            dataset['Upper'][i+2] =  round(dataset['Forecast'][i+2] + 
                                           (tt *(dataset['Std'][i+2] / math.sqrt(nn))))
            
            dataset['Lower'][i+2] =  round(dataset['Forecast'][i+2] - 
                                           (tt *(dataset['Std'][i+2] / math.sqrt(nn))))
            
            if (dataset['Lower'][i+2] < 0):
                dataset['Lower'][i+2]  = 0                
            
            
            i = i+3
            
                    
            
    else:
        i = i+1
    

<ipython-input-227-e2e3100823ed>:108: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataset['Forecast'][i+3] = round((dataset['Sales'][i] +
<ipython-input-227-e2e3100823ed>:114: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataset['Std'][i+3] = stdd
<ipython-input-227-e2e3100823ed>:116: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataset['Upper'][i+3] =  round(dataset['Forecast'][i+3] +
<ipython-input-227-e2e3100823ed>:119: SettingW

In [228]:
dataset['IW'] = dataset['Initial_Stock'] * 5
#OUTPUT PRINTING
dataset = dataset.drop(['Std'],axis = 1)

print('Execution Time is : ', (time.time()-start_time), " seconds")

max_value = dataset['Weeks'].max()
gurobi_inputs = dataset[dataset.Weeks==max_value]

Execution Time is :  138.5656337738037  seconds


In [229]:
gurobi_inputs.tail()

,Stores,SKU,Weeks,Category,Initial_Stock,Sales,Price,Wmin,Wmax,IW,Forecast,Lower,Upper
10705,UA20,7K8076Z8_D3W_XS,201749,KEY TRIKO KAZAK,2,0,50.501161,2,6,10,1,1,1
10707,UA20,7K9189Z8_A1J_L,201749,KEY TRIKO KAZAK,1,0,54.297856,2,6,5,1,1,1
10709,UA20,7K9189Z8_A1J_M,201749,KEY TRIKO KAZAK,2,0,54.297856,2,6,10,1,0,10
10711,UA20,7KI586Z8_HFH_L,201749,KEY TRIKO KAZAK,3,0,49.593361,2,6,15,2,2,2
10713,UA20,7KI586Z8_HFH_M,201749,KEY TRIKO KAZAK,3,0,49.593361,2,6,15,2,2,2


-----

In [ ]:
def set_Upper(Data):
    
    df = Data
    
    Opt = df['SKU'].str.split('_', n = 2, expand = True)
    Opt = Opt[0]+'_'+Opt[1]
    df['Option']= Opt
    
    for i in df['Option'].unique():
    
        for j in df.loc[df['Option'] == i]['Stores'].unique():
            size_list = []

            for k in df.loc[(df['Option'] == i) & (df['Stores'] == j)]['SKU']:
                SKU_lower = int(df.loc[(df['Option'] == i) & (df['Stores'] == j) & (df['SKU'] == k)]['Lower'])
                SKU_upper = int(df.loc[(df['Option'] == i) & (df['Stores'] == j) & (df['SKU'] == k)]['Upper'])

                if ((SKU_lower == 0) and (SKU_upper == 0)):
                    size_list.append(k)

            if len(size_list) < 4:
                for t in size_list:
                    index = df.loc[(df['Stores'] == j) & (df['SKU'] == t)].index[0]
                    df['Upper'][index] = 2
                    
    df.pop('Option')
    df = df.reset_index()  
    return df

In [230]:
gurobi_inputs = set_Upper(gurobi_inputs)

<ipython-input-156-706887d17c6a>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Option']= Opt
<ipython-input-156-706887d17c6a>:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Upper'][index] = 2
<ipython-input-230-2865402c1dd4>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  gurobi_inputs = set_Upper(gurobi_inputs)


----

In [105]:
def Accurate_Assortment(data):
    df = data

    Opt = df['SKU'].str.split('_', n = 2, expand = True)
    Opt = Opt[0]+'_'+Opt[1]

    size = df['SKU'].str.split('_', n = 2, expand = True)
    size = size[2]

    df['Option']= Opt
    df['Size'] = size

    SKU = []
    Price = []
    Category = []
    Store = []

    for i in df['Option'].unique():

        for j in df.loc[df['Option'] == i]['Stores'].unique():
            Size = ['XXS','XS','S','M','L','XL','XXL']

            for k in df.loc[(df['Option'] == i) & (df['Stores'] == j)]['Size'].unique():
                Size.remove(k)

            for t in Size:
                missing_SKU = i+'_'+t
                SKU_Price = df.loc[df['Option'] == i].reset_index()['Price'][0]
                SKU_Category = df.loc[df['Option'] == i].reset_index()['Category'][0]
                SKU_Store = j

                SKU.append(missing_SKU)
                Price.append(SKU_Price)
                Category.append(SKU_Category)
                Store.append(SKU_Store)

    missing_SKUs = pd.DataFrame(list(zip(SKU,Price,Category,Store)), columns = ['SKU','Price','Category','Stores'])

    # Assign Wmin and Wmax by Size
    missing_SKUs["Wmin"] = np.nan
    missing_SKUs["Wmax"] = np.nan

    for i in range(0,len(missing_SKUs)):
        if (missing_SKUs['SKU'][i][-3:] == 'XXL' or missing_SKUs['SKU'][i][-3:] == 'XXS'):
            missing_SKUs['Wmin'][i] = 1
            missing_SKUs['Wmax'][i] = 3
        else:
            missing_SKUs['Wmin'][i] = 2
            missing_SKUs['Wmax'][i] = 6

    # Assign dummy columns
    missing_SKUs['Weeks'] =  df['Weeks'][0]
    missing_SKUs['Initial_Stock'] = 0
    missing_SKUs['Sales'] = 0
    missing_SKUs['IW'] = 20
    missing_SKUs['Forecast'] = 0
    missing_SKUs['Lower'] = 0
    missing_SKUs['Upper'] = 2

    # Reorder the columns: Same with Gurobi_Inputs.xlsx
    missing_SKUs = missing_SKUs[['Stores','SKU','Weeks','Category','Initial_Stock',
                                 'Sales','Price','Wmin','Wmax','IW','Forecast','Lower','Upper']]

    df.pop('Option')
    df.pop('Size')

    frames = [missing_SKUs, df]
    df = pd.concat(frames)
    df = df.reset_index()
    df.pop('index')
    df.pop('level_0')
    
    return df

In [231]:
gurobi_inputs = Accurate_Assortment(gurobi_inputs)

<ipython-input-105-da43f3cca20f>:45: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  missing_SKUs['Wmin'][i] = 1
<ipython-input-105-da43f3cca20f>:46: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  missing_SKUs['Wmax'][i] = 3
<ipython-input-105-da43f3cca20f>:48: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  missing_SKUs['Wmin'][i] = 2
<ipython-input-105-da43f3cca20f>:49: SettingWithCopyWarning: 
A value is trying to be set on a copy of a s

In [232]:
for i in range(0,len(gurobi_inputs)):
    if gurobi_inputs['Lower'][i] == gurobi_inputs['Upper'][i]:
        gurobi_inputs['Upper'][i] += 2

<ipython-input-232-72763fb6360b>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  gurobi_inputs['Upper'][i] += 2
